In [1]:
import pandas as pd
from google.cloud import bigquery 
from google.oauth2 import service_account
from google.cloud import storage
import openpyxl
import numpy as np
import time
from datetime import datetime

# Carga las credenciales desde el archivo JSON y asigna los alcances
credentials = service_account.Credentials.from_service_account_file(
    'Cuenta.json',
)
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

# Guardar el tiempo de inicio
start_time = time.time()
hora_inicio = datetime.now()

# Imprime la hora de inicio
print("Hora de inicio:", hora_inicio.strftime("%Y-%m-%d %H:%M:%S"))

#Variables Manuales

filiales = ['VANTI','CUNDINAMARCA','BOYACA','GASORIENTE', 'BARRANCABERMEJA', 'GASNACER']

proyeccion_anterior = {
    'filial': filiales,
    'proyeccion_anterior': [65362,8404,4036,6578,1576,3062]
}

# proyeccion_anterior = []
# # Solicitar entrada del usuario para cada filial
# for filial in filiales:
#     valor = int(input(f"Ingrese la cantidad de la proyección anterior para la filial {filial}: "))
#     proyeccion_anterior.append(valor)

df_proyeccion_anterior = pd.DataFrame(proyeccion_anterior)

marketing = {
    'filial': filiales,
    'marketing': [0,0,0,0,0,0]
}

# marketing = []
# # Solicitar info marketing
# for filial in filiales:
#     valor_marketing = int(input(f"Ingrese la cantidad de marketing para la filial {filial}: "))
#     marketing.append(valor_marketing)

df_marketing = pd.DataFrame(marketing)

capacidad_operativa = {
    'filial': filiales,
    'capacidad_operativa': [0,0,0,0,0,0]
}

# capacidad_operativa = []
# # Solicitar info marketing
# for filial in filiales:
#     valor_cp = int(input(f"Ingrese las operaciones que se dejan de hacer por falta de capacidad operativa para la filial {filial}: "))
#     capacidad_operativa.append(valor_cp)

df_capacidad_operativa = pd.DataFrame(capacidad_operativa)

print(f'Variables Manuales cargadas')

query = """
SELECT * FROM datalake-vanti.OOCC.Proyeccion_Demanda_py
"""
df_data = client.query(query).to_dataframe()
hora_demanda = datetime.now()
print("Demanda Cargada Hora:", hora_inicio.strftime("%Y-%m-%d %H:%M:%S"))

Perdida_Mercado = """
SELECT * FROM datalake-vanti.OOCC.Proyeccion_Perdida_mercado_py
"""
df_PM = client.query(Perdida_Mercado).to_dataframe()
hora_Perdida_Mercado = datetime.now()
print("Perdida de Mercado Cargada Hora:", hora_Perdida_Mercado.strftime("%Y-%m-%d %H:%M:%S"))

Programadas = """
SELECT filial,Count(*) Programadas
FROM datalake-vanti.OOCC.VW_Efectividad_Total
where Sub_proceso = 'Revision Periodica'
and format_date('%Y-%m-01',Fe_comunicacion2) = format_date('%Y-%m-01', date_sub(current_date(),interval 1 month))
group by 1
"""
df_Programadas = client.query(Programadas).to_dataframe()
hora_Programadas = datetime.now()
print("Programadas Cargada Hora:", hora_Programadas.strftime("%Y-%m-%d %H:%M:%S"))

Visitas_Certificacion = """
SELECT * FROM datalake-vanti.OOCC.Proyeccion_Visitas_Cert_py
"""
df_ctd_vis_cer = client.query(Visitas_Certificacion).to_dataframe()
hora_Visitas_Certificacion = datetime.now()
print("Visitas Certificacion Cargada Hora:", hora_Visitas_Certificacion.strftime("%Y-%m-%d %H:%M:%S"))

Visitas_neje = """
SELECT * FROM datalake-vanti.OOCC.Proyeccion_visitas_NEJE_py
"""
df_neje = client.query(Visitas_neje).to_dataframe()
hora_Visitas_neje = datetime.now()
print("No Efectivas Cargada Hora:", hora_Visitas_neje.strftime("%Y-%m-%d %H:%M:%S"))

Efectivas = """
SELECT filial, round(1-No_Efectivas,3) Efectivas FROM datalake-vanti.OOCC.Proyeccion_visitas_NEJE_py
"""
df_Efectivas = client.query(Efectivas).to_dataframe()
hora_Efectivas = datetime.now()
print("Efectivas Cargada Hora:", hora_Efectivas.strftime("%Y-%m-%d %H:%M:%S"))

vtr = """
SELECT * FROM datalake-vanti.OOCC.Proyeccion_VTR_py
"""
df_vtr = client.query(vtr).to_dataframe()
hora_vtr = datetime.now()
print("VTR Cargada Hora:", hora_vtr.strftime("%Y-%m-%d %H:%M:%S"))

Ausente_cer_ex = """
SELECT * FROM datalake-vanti.OOCC.Proyeccion_Ausentes_cer_ex_py
"""
df_Ausente_cer_ex = client.query(Ausente_cer_ex).to_dataframe()
hora_Ausente_cer_ex = datetime.now()
print("Ausentes Cargada Hora:", hora_Ausente_cer_ex.strftime("%Y-%m-%d %H:%M:%S"))

Defectos_cer_ex = """
SELECT * FROM datalake-vanti.OOCC.Proyeccion_defectos_cer_ex_py
"""
df_Defectos_cer_ex = client.query(Defectos_cer_ex).to_dataframe()
hora_Defectos_cer_ex = datetime.now()
print("Defectos externos Cargada Hora:", hora_Defectos_cer_ex.strftime("%Y-%m-%d %H:%M:%S"))

Citas_Incumplidas = """
SELECT * FROM datalake-vanti.OOCC.Citas_Incumplidas
"""
df_Citas_Incumplidas = client.query(Citas_Incumplidas).to_dataframe()
hora_Citas_Incumplidas = datetime.now()
print("Citas Incumplidas Cargada Hora:", hora_Citas_Incumplidas.strftime("%Y-%m-%d %H:%M:%S"))

Espontanea_RYM = """
SELECT * FROM datalake-vanti.OOCC.Espontanea_RYM
"""
df_Espontanea_RYM = client.query(Espontanea_RYM).to_dataframe()
hora_Espontanea_RYM = datetime.now()
print("Espontanea Cargada Hora:", hora_Espontanea_RYM.strftime("%Y-%m-%d %H:%M:%S"))

mes_60 = """
SELECT * FROM datalake-vanti.OOCC.mes_60
"""
df_mes_60 = client.query(mes_60).to_dataframe()
hora_mes_60 = datetime.now()
print("Mes 60 Cargada Hora:", hora_mes_60.strftime("%Y-%m-%d %H:%M:%S"))

defectos = """
SELECT * FROM datalake-vanti.OOCC.Proyeccion_Defectos_py
"""
df_defectos = client.query(defectos).to_dataframe()
hora_defectos = datetime.now()
print("Defectos Cargada Hora:", hora_defectos.strftime("%Y-%m-%d %H:%M:%S"))

df_final =  pd.DataFrame(df_data)
#df_final = df_final.groupby(['filial', 'Year_Mes'], as_index=False)['Demanda'].sum(),['Vencimientos'].avg()
df_final = df_final.groupby(['filial', 'Year_Mes'], as_index=False).agg({'Demanda': 'sum', 'Vencimientos': 'mean'})

df_final = df_final.merge(df_PM, on='filial', how='left')
df_final = df_final.merge(df_ctd_vis_cer, on='filial', how='left')
df_final = df_final.merge(df_neje, on='filial', how='left')
df_final = df_final.merge(df_vtr, on='filial', how='left')
df_final = df_final.merge(df_Ausente_cer_ex, on='filial', how='left')
df_final = df_final.merge(df_Defectos_cer_ex, on='filial', how='left')
df_final = df_final.merge(df_Citas_Incumplidas, on='filial', how='left')
df_final = df_final.merge(df_Espontanea_RYM, on='filial', how='left')
df_final = df_final.merge(df_mes_60, on='filial', how='left')
df_final = df_final.merge(df_Efectivas, on='filial', how='left')
df_final = df_final.merge(df_defectos, on='filial', how='left')
df_final = df_final.merge(df_Programadas, on='filial', how='left')
df_final = df_final.merge(df_proyeccion_anterior, on='filial', how='left')
df_final = df_final.merge(df_marketing, on='filial', how='left')
df_final = df_final.merge(df_capacidad_operativa, on='filial', how='left')

print("Merge Cargados Hora:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

condicion = (df_final['filial'].isin(['VANTI', 'CUNDINAMARCA']))

# Convertir la columna 'Year_Mes' a datetime
df_final['Year_Mes'] = pd.to_datetime(df_final['Year_Mes'])

# Obtener el mes actual
mes_actual = pd.Timestamp.now().to_period('M')
fecha_proyeccion = pd.Timestamp.now()
fecha_mod = df_final['Year_Mes'].dt.to_period('M')

# Definir la condición
condicion_fecha = (fecha_mod== mes_actual)

#f_final['fecha_mod'] = fecha_mod

# Multiplicar columnas
df_final['Desviacion'] = round((df_final['Programadas'] + df_final['capacidad_operativa'] -df_final['proyeccion_anterior'] ) / df_final['proyeccion_anterior'], 1)
df_final['A'] = round(df_final['Demanda'] * df_final['perdida_mercado'], 1)
df_final['B'] = df_final['VTR']
df_final['C'] = round((df_final['Demanda'] - df_final['A'] + df_final['VTR']) * df_final['No_Efectivas'], 1)
df_final['D'] = round(df_final['C'] * df_final['No_Efectivos_certificado'], 1)
df_final['E'] = round((df_final['Demanda'] - df_final['A']) * df_final['Efectivas'] * df_final['Defectos_Certificados'], 1)
df_final['F'] = round(df_final['C'] * df_final['Citas_Incumplidas'], 1)
df_final['G'] = df_final['marketing']
df_final['H'] = round(df_final['Desviacion'] * df_final['proyeccion_anterior'], 1)
df_final['I'] = df_final['Mes_60']
df_final['I'] = np.where(condicion_fecha,df_final['Mes_60'],0)
df_final['G'] = np.where(condicion_fecha, df_final['G'],0)
df_final['H'] = np.where(condicion_fecha, df_final['H'],0)
df_final['I'] = np.where(condicion_fecha, df_final['I'],0)
df_final['J'] = df_final['Espontanea']

formula_k_condicion = (df_final['Demanda'] - df_final['A'] + df_final['VTR'] + df_final['D'] + df_final['F']) * df_final['Visitas_Certificacion']
formula_k_sin_condicion = (df_final['Demanda'] - df_final['A'] + df_final['VTR'] + df_final['D'] + df_final['F'] + df_final['G'] + df_final['Mes_60']) * df_final['Visitas_Certificacion']

df_final['K'] = np.where(condicion_fecha, formula_k_condicion.round(1), formula_k_sin_condicion.round(1))
df_final['Desviacion'] = np.where(condicion_fecha, df_final['Desviacion'],0 )

# Calcular la suma según la condición
df_final['Proyeccion'] = np.where(condicion,
                                  df_final['Demanda'] - df_final['A'] + df_final['B'] + df_final['C'] + df_final['D'] + df_final['E'] + df_final['F'] + df_final['G'] + df_final['H'] + df_final['I'] + df_final['J'] + df_final['K'],
                                  df_final['Demanda'] - df_final['A'] + df_final['B'] + df_final['C'] + df_final['F'] + df_final['G'] + df_final['H'] + df_final['K'])  # Define la otra suma que quieres hacer cuando la condición no se cumple

# Convertir la columna datetime64[ns] a date
df_final['Year_Mes'] = df_final['Year_Mes'].dt.date

# Convertir la columna datetime64[ns] a date
df_final['fecha_proyeccion'] = fecha_proyeccion

# Escribir el DataFrame final en un archivo Excel
df_final.to_excel('Proyeccion_Final.xlsx', index=False)

# Nombre de la tabla de destino en BigQuery
nombre_tabla_destino = 'datalake-vanti.OOCC.Proyeccion_final'

# Calcular y mostrar el tiempo de ejecución
end_time = time.time()
execution_time = end_time - start_time
    
print(f"Tiempo de ejecución: {execution_time:.2f} segundos")
print("Archivo Excel 'Proyeccion_Final.xlsx' creado exitosamente.")
# Verificar los tipos de datos de las columnas
#print(df_final.dtypes)

# Cargar el DataFrame de proyección en la tabla de destino
job = client.load_table_from_dataframe(df_final, nombre_tabla_destino)
job.result()  # Esperar a que la carga se complete
print(f"Datos de proyección cargados exitosamente en {nombre_tabla_destino}")

Hora de inicio: 2025-04-01 11:09:06
Variables Manuales cargadas
Demanda Cargada Hora: 2025-04-01 11:09:06
Perdida de Mercado Cargada Hora: 2025-04-01 11:10:12
Programadas Cargada Hora: 2025-04-01 11:10:42
Visitas Certificacion Cargada Hora: 2025-04-01 11:10:46
No Efectivas Cargada Hora: 2025-04-01 11:11:14
Efectivas Cargada Hora: 2025-04-01 11:11:39
VTR Cargada Hora: 2025-04-01 11:11:42
Ausentes Cargada Hora: 2025-04-01 11:12:13
Defectos externos Cargada Hora: 2025-04-01 11:13:21
Citas Incumplidas Cargada Hora: 2025-04-01 11:13:51
Espontanea Cargada Hora: 2025-04-01 11:13:55
Mes 60 Cargada Hora: 2025-04-01 11:14:09
Defectos Cargada Hora: 2025-04-01 11:14:58
Merge Cargados Hora: 2025-04-01 11:14:58
Tiempo de ejecución: 352.25 segundos
Archivo Excel 'Proyeccion_Final.xlsx' creado exitosamente.
Datos de proyección cargados exitosamente en datalake-vanti.OOCC.Proyeccion_final
